In [ ]:
For each file.zip
	download to loca_folder

For each file.zip in local_folder
	Unzip

For each file.zip
	Iterate in pandas (certificates.csv)
        filter London local authorities
        Add missing columns
        order columns
        save as parquet in local
        send to datalake
        clean parquet from local
    clear files

In [ ]:
import requests
import zipfile
import pandas as pd
import os
import pickle

token = os.environ['EPC_TOKEN']

In [ ]:


with open("../../../../data/epc-columns.pkl", "rb") as f:
    columns = pickle.load(f)
columns_set = set(columns)

In [8]:
import requests
import os
import pickle
# import configs

epc_file_path = "../../../../data/epc-files.pkl"
with open(epc_file_path, "rb") as f:
    epc_files = pickle.load(f)
    
token = os.environ['EPC_TOKEN']

def download_epc_file(local_folder, file, chunk_size):

    if not os.path.exists(local_folder):
        os.makedirs(local_folder)

    output_file = f"{local_folder}/{file}"
    url = f"https://epc.opendatacommunities.org/api/v1/files/{file}"
    headers = {
        "Authorization": f"Basic {token}"
    }
    
    try:
        response = requests.get(url, headers=headers, stream=True)
        response.raise_for_status()
    
        with open(output_file, "wb") as out_file:
            for chunk in response.iter_content(chunk_size):
                out_file.write(chunk)
        print(f"{file} downloaded.")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")


def download_epc_file_list(local_folder, file_list, chunk_size):
    for file in file_list:
        download_epc_file(local_folder, file, chunk_size)

In [12]:
# epc_files = epc_files[:3]
# epc_files

['display-2024-03.zip', 'domestic-2021.zip', 'domestic-2023.zip']

In [47]:
local_folder = 'prova_epc/'
chunk_size = 100_000

download_epc_file_list(local_folder, epc_files, chunk_size)

display-2024-03.zip downloaded.
domestic-2021.zip downloaded.
domestic-2023.zip downloaded.


In [48]:
import zipfile
import os

def extract_file(local_folder, file):
    zipped_file = f"{local_folder}{file}"
    extract_to = f"{zipped_file}"[:-4]
    try:
        with zipfile.ZipFile(zipped_file, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
    except Exception as e:
        print(f"Error: {e}")

def extract_file_list(local_folder, file_list):
    for file in file_list:
        extract_file(local_folder, file)
        os.remove(f"{local_folder}{file}")

In [50]:
local_folder = 'prova_epc/'

extract_file_list(local_folder, epc_files)

Error: [Errno 2] No such file or directory: 'prova_epc/display-2024-03.zip'


FileNotFoundError: [Errno 2] No such file or directory: 'prova_epc/display-2024-03.zip'

In [32]:
extract_to = f"{zipped_file}"[:-4]

extract_to

'prova_epc/display-2024-03'

In [46]:
!rm -rf prova

In [187]:
import pandas as pd
from bucket_utils import Bucket

# def transform(df):
    

def stream_file_to_bucket(bucket_session,
                          local_folder, 
                          local_filename,
                          destination_folder,
                          chunk_size):

    df_iter = pd.read_csv(f"{local_folder}{local_filename}", iterator=True, chunksize=chunk_size,low_memory=False)

        # transform
            # filter London local authorities
            # Add missing columns
            # order columns
        # save as parquet in local
        # send to datalake
        # clean parquet from local

    for i, chunk in enumerate(df_iter):
        chunk_file_name = f'{local_filename[:-4]}_{i}.parquet'
        local_file_path = f"{local_folder}{chunk_file_name}"
        # chunk = transform(chunk)
        if len(chunk)>0:
            chunk.to_parquet(local_file_path)
            with open(local_file_path, 'rb') as file:
                bucket_session.put_object(local_folder, chunk_file_name, 
                                          destination_folder, chunk_file_name)
            os.remove(local_file_path)

def stream_epc_files_to_bucket(file_list, 
                               region, 
                               namespace, 
                               bucket_name,
                               epc_local_folder,
                               epc_bucket_folder,
                               chunk_size):

    bucket_session = Bucket(region, namespace, bucket_name)
    
    for file in file_list:
        epc_unzipped_file_folder = file+"/"
        local_folder = f"{epc_local_folder}{epc_unzipped_file_folder}"
        stream_file_to_bucket(bucket_session = bucket_session,
                              local_folder = local_folder, 
                              local_filename = 'certificates.csv',
                              destination_folder = f"{epc_bucket_folder}{local_folder.split('/')[1]}/",
                              chunk_size=chunk_size)

    bucket_session.close()

In [191]:
file_list = ['display-2024-03', 'domestic-2021','domestic-2023']

OCI_REGION ='uk-london-1'
OCI_NAMESPACE = 'lrqgbz9z6zlj'
bucket_name = 'london-property-sales-price'
epc_local_folder = "prova_epc/"
epc_bucket_folder = "epc_downloads/"
chunk_size = 100_000

stream_epc_files_to_bucket(file_list, 
                            OCI_REGION, 
                            OCI_NAMESPACE, 
                            bucket_name,
                            epc_local_folder,
                            epc_bucket_folder,
                            chunk_size)

In [182]:
OCI_REGION ='uk-london-1'
OCI_NAMESPACE = 'lrqgbz9z6zlj'
bucket_name = 'london-property-sales-price'

epc_local_folder = "prova/"
epc_bucket_folder = "epc_downloads/"

epc_unzipped_file_folder = 'domestic-2021'+"/"



local_folder = f"{epc_local_folder}{epc_unzipped_file_folder}"
destination_folder = f"{epc_bucket_folder}{local_folder.split('/')[1]}/"

chunk_size = 100_000
# local_folder.split("/")[0]

In [167]:
destination_folder

'epc_downloads/domestic-2021/'

In [ ]:
bucket_session = Bucket(OCI_REGION, OCI_NAMESPACE, bucket_name)

In [183]:
stream_file_to_bucket(bucket_session = bucket_session,
                      local_folder = local_folder, 
                      local_filename = 'certificates.csv',
                      destination_folder = f"{epc_bucket_folder}{local_folder.split('/')[1]}/",
                      chunk_size=chunk_size)

In [194]:
bucket_session.list_objects()

No content to list.


In [193]:
bucket_session.delete_folder_content("")

All objects in the folder '' have been deleted.


In [170]:
local_filename = 'certificates.csv'

local_folder = f"{epc_local_folder}{epc_unzipped_file_folder}"

In [195]:
df_iter = pd.read_csv("prova_epc/domestic-2021/certificates.csv", iterator=True, chunksize=chunk_size, low_memory=False)

In [196]:
df = next(it)

In [ ]:
# transform
    # filter London local authorities
    # Add missing columns
    # order columns
# save as parquet in local
# send to datalake
# clean parquet from local

In [197]:
df

,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,...,LOCAL_AUTHORITY_LABEL,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE
100000,714f575b44bb8e2203d5caea6dfc72ba90cad02340c2ee...,Flat 5,26 Ickburgh Road,Hackney,E5 8AD,10001614448,D,C,58,69,...,Hackney,Hackney North and Stoke Newington,LONDON,England and Wales: 1900-1929,2021-08-12 08:19:34,Rented (private),10.0,NaN,1.000210e+11,Energy Assessor
100001,714fc2c8454f3e87ef8be0d3a1fe1ffdecb584a5626708...,149 BROUGHTON AVENUE,NaN,NaN,HP20 1PU,10001366252,D,B,64,88,...,Buckinghamshire,Aylesbury,AYLESBURY,England and Wales: 1967-1975,2021-07-26 20:34:22,Rented (private),8.0,NaN,7.660265e+08,Energy Assessor
100002,714ea2dda5389a5b3a8819b6f2eaacd067fe7cb8d4fba1...,30 Westford Grange,Westford,NaN,TA21 0EP,2287594568,C,B,74,84,...,Somerset West and Taunton,Taunton Deane,WELLINGTON,England and Wales: 2007-2011,2021-08-14 14:03:20,Rented (social),15.0,NaN,1.001426e+10,Energy Assessor
100003,714eaab90964327d7e3365fc4202b272bcf2ffdcb450a7...,16 WHICKHAM CLOSE,NaN,NaN,WA8 9ZQ,10001379562,C,B,75,84,...,Halton,Halton,WIDNES,England and Wales: 1996-2002,2021-07-30 13:46:09,Owner-occupied,21.0,NaN,1.001061e+10,Energy Assessor
100004,714f0f2c49066636406f6fcfa4b4826b8f36fa0c5cafb3...,3 Spenlows Road,Bletchley,NaN,MK3 6JT,10001508001,D,B,65,84,...,Milton Keynes,Milton Keynes South,MILTON KEYNES,England and Wales: 1976-1982,2021-09-14 08:44:30,Owner-occupied,7.0,NaN,2.505189e+07,Energy Assessor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,ac0e255411c469e2ac2333f50996d521a77501810fe7d6...,19 Beaumaris Road,Sawtry,NaN,PE28 5SF,10001403607,E,B,53,85,...,Huntingdonshire,North West Cambridgeshire,HUNTINGDON,England and Wales: 1983-1990,2021-09-17 09:28:30,Rented (social),6.0,NaN,1.000901e+11,Energy Assessor
199996,ac10198134ae0972f0f329b3789570d5afd6175c3e2642...,3 Puddle Lane,St. Martins,NaN,SY11 3DE,10001463345,C,B,69,90,...,Shropshire,North Shropshire,OSWESTRY,England and Wales: 1976-1982,2021-09-22 14:31:58,Rented (private),7.0,NaN,1.000700e+11,Energy Assessor
199997,ac105d5546fde66ffc0a73be4e71f21becb0ce2cf62cd1...,1 WISHART GREEN,OLD FARM PARK,NaN,MK7 8QB,10001343303,C,B,71,81,...,Milton Keynes,Milton Keynes South,MILTON KEYNES,England and Wales: 1991-1995,2021-08-08 16:18:16,Owner-occupied,19.0,NaN,2.508708e+07,Energy Assessor
199998,ac12786f2cec371d6b2940e43f3dbb103c371059fbd568...,61 Wood Grove,Silver End,NaN,CM8 3FN,10001566891,C,B,76,87,...,Braintree,Witham,WITHAM,England and Wales: 2007-2011,2021-09-10 21:41:36,Owner-occupied,15.0,NaN,1.001212e+10,Energy Assessor


In [ ]:
def transform(df):
    

In [145]:
df.to_parquet('pr.parquet')

In [146]:
with open('pr.parquet', 'rb') as file:
    bucket_session.put_object("", "pr.parquet", 
                              "prova/", "pr.parquet")

In [ ]:
## Filter London authorities
df['LOCAL_AUTHORITY']

In [ ]:
# Add missing columns

cols_in_file_set = set(df.columns)
missing_cols = columns_set - cols_in_file_set

for col in missing_cols:
    df[col] = None

In [ ]:
# order columns
df = df[[columns]]